In [1]:
%load_ext autoreload
%autoreload 2
import enoki as ek
import mitsuba
mitsuba.set_variant('gpu_autodiff_rgb')


import time
import os
import glob
import torch
import cv2
import copy
import random
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm
import wandb
from mitsuba.python.autodiff import render, render_torch, write_bitmap
from mitsuba.python.util import traverse
from mitsuba.core.xml import load_file
from mitsuba.core import Thread, Vector3f,LogLevel


from torch_openpose.body import Body
from torch_openpose import util



In [2]:
# FUNCTIONS
def wandb_init(name):
    run = wandb.init(
        project = "mts_mutliple_images", 
        reinit = True,
        name = name,
        config={
            "epochs": 5,
            "batch_size": 4,
            "noise_width": 200,
            "noise_height": 400,
        }
    )
    
    return run 

def get_scenes_filenames(directory):
    scenes = []
    for file in tqdm(glob.glob(f"{directory}/*.xml")):
        scenes.append(file)
        
    # shuffel scenes due to naming 
    random.seed(42)
    random.shuffle(scenes)
    
    return scenes

def get_candidate_from_tensor(rendering_torch, body_estimation):
    PERMUTE_BGR = [2, 1, 0]
     # make torch RGB to BGR and permute and stack
    rendering_torch_BGR = rendering_torch[:, :, PERMUTE_BGR]
    rendering_torch_input = torch.stack([rendering_torch_BGR.permute((2, 0, 1))  - 0.5])
    
    # Render a reference image (no derivatives used yet)
    heatmap_avg, paf_avg = body_estimation.compute_heatmap_paf_avg(rendering_torch_input, ORIG_SHAPE)
    candidate, subset = body_estimation.get_pose(heatmap_avg, paf_avg, ORIG_SHAPE)
    rendering_torch_np =  rendering_torch[:, :, PERMUTE_BGR].detach().cpu().numpy()
    canvas = copy.deepcopy(rendering_torch_np)
    canvas = util.draw_bodypose(canvas, candidate, subset)
    
    return canvas, candidate, subset

def get_image_and_score(scene_path, params_torch, body_estimation, diff_parameter, spp=8):
    thread = Thread.thread()
    thread.file_resolver().append(test_dir)
    logger = thread.logger()
    logger.set_log_level(LogLevel.Warn)
    
    # load scene
    scene = load_file(scene_path)
    params = traverse(scene)
    rendering_torch = render_torch(scene, spp=spp)
    canvas_orig, candidate_orig, subset_orig = get_candidate_from_tensor(rendering_torch, body_estimation)
   
    
    # update the params with the new noise value
    params[diff_parameter] = params_torch[diff_parameter].detach().cpu().numpy()
    params.update()
    
    rendering_torch = render_torch(scene, params=params, unbiased=True, spp=spp, **params_torch)
    canvas_noise, candidate_noise, subset_noise = get_candidate_from_tensor(rendering_torch, body_estimation)
    
    metric = pose_loss_single_human(subset_noise, subset_orig)
    return canvas_orig[:, :, [2, 1, 0]], canvas_noise[:, :, [2, 1, 0]], metric


def pose_loss_single_human(newHuman, oldHuman):
    if len(oldHuman) == 0 or len(newHuman) == 0:
        return 0
    new_detected = 0
    old_detected = 0
    for part in range(18):
        if newHuman[0][part] != -1:
            new_detected += 1
        if oldHuman[0][part] != -1:
            old_detected += 1
    return new_detected / old_detected



In [3]:
#CONSTS
STRIDE = 8
PAD_VALUE = 128
IMAGE_WIDTH, IMAGE_HEIGHT = 432, 368
ORIG_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, 3)
snapshot_name = f'../snapshots/mts_noise.npy'
train_dir = '../data/allTransforms/train'
test_dir = '../data/allTransforms/test'

train_scenes = get_scenes_filenames(train_dir)
test_scenes = get_scenes_filenames(test_dir)


100%|██████████| 200/200 [00:00<00:00, 866591.74it/s]


In [4]:
body_estimation = Body('/home/ax/data/programs/pytorch-openpose/model/body_pose_model.pth', True)
# set defaults
body_estimation.imageToTest_padded_shape = ORIG_SHAPE
body_estimation.pad = [0, 0, 0, 0]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

thread = Thread.thread()
thread.file_resolver().append('pose_scene')
logger = thread.logger()
logger.set_log_level(LogLevel.Warn)

scene = load_file('pose_scene/scene.xml')

# Find differentiable scene parameters
params = traverse(scene)
print(len(params['rect.bsdf.reflectance.data']))


cuda:0
240000


In [5]:
# Render a reference image (no derivatives used yet)
"""
rendering_torch = render_torch(scene, spp=32)

# make torch RGB to BGR
PERMUTE_BGR = [2, 1, 0]
rendering_torch_BGR = rendering_torch[:, :, PERMUTE_BGR]
# permute and stack
rendering_torch_input = torch.stack([rendering_torch_BGR.permute((2, 0, 1))  - 0.5])

canvas = rendering_torch_BGR.cpu().detach().numpy() 
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)
ax.imshow(canvas[:, :, [2, 1, 0]])
ax.axis('off')
"""

"\nrendering_torch = render_torch(scene, spp=32)\n\n# make torch RGB to BGR\nPERMUTE_BGR = [2, 1, 0]\nrendering_torch_BGR = rendering_torch[:, :, PERMUTE_BGR]\n# permute and stack\nrendering_torch_input = torch.stack([rendering_torch_BGR.permute((2, 0, 1))  - 0.5])\n\ncanvas = rendering_torch_BGR.cpu().detach().numpy() \nfig = plt.figure(figsize=(10, 10))\nax = fig.add_subplot(1, 1, 1)\nax.imshow(canvas[:, :, [2, 1, 0]])\nax.axis('off')\n"

In [ ]:
run = wandb_init('multipleImage')
noise_resolution  = np.array([200, 400])
steps = len(train_scenes) // run.config.batch_size

diff_parameter = 'rect.bsdf.reflectance.data'

# Which parameters should be exposed to the PyTorch optimizer?
params.keep([diff_parameter])
params.update()
params_torch = params.torch() # main pytorch params

# Construct a PyTorch Adam optimizer that will adjust 'params_torch'
opt = torch.optim.Adam(params_torch.values(), lr=.2)
objective = torch.nn.MSELoss(reduction='sum')

# target
zero_heatmap = torch.zeros((run.config.batch_size, 19, 46, 54), device=device)
zero_paf = torch.zeros((run.config.batch_size, 38, 46, 54), device=device)

myThread = Thread.thread()
myThread.file_resolver().append(train_dir)
logger = myThread.logger()
logger.set_log_level(LogLevel.Warn)

global_step = 0
for epoch in range(run.config.epochs):
    print(f"run {epoch + 1}/{run.config.epochs}")
    for step in tqdm(range(steps)):
        if step % 50 ==  0:
            write_bitmap('output/universal_noise_%03i.png' % step, params_torch[diff_parameter], noise_resolution)
            np.save(snapshot_name, params_torch[diff_parameter].detach().cpu().numpy())
        opt.zero_grad()
        body_estimation.model.zero_grad()

        # list of all rendering from this batch
        rendering_batch = []

        for i in range(step * run.config.batch_size, (step * run.config.batch_size) + run.config.batch_size):
            scene = load_file(train_scenes[i])
            params = traverse(scene)
            params.keep([diff_parameter])

            # update the diffparamter
            params[diff_parameter] = params_torch[diff_parameter].detach().cpu().numpy()

            params.update()
            # render image
            # maybe add malloc_trim=True if memory issues
            cur_rednering_torch = render_torch(scene, params=params, unbiased=True, spp=8, **params_torch)

            # transform from rgb to bgr
            PERMUTE_BGR = [2, 1, 0]
            rendering_torch_BGR = cur_rednering_torch[:, :, PERMUTE_BGR]
            rendering_batch.append(rendering_torch_BGR.permute((2, 0, 1))  - 0.5)

        # permute channels to pytorch order and stack
        rendering_torch_input = torch.stack(rendering_batch)

        # compute the avg heatmap and paf map
        paf, heatmap = body_estimation.model(rendering_torch_input)


        ob_val_heat = objective(heatmap, zero_heatmap)
        ob_val_paf = objective(paf, zero_paf)
        ob_val = ob_val_heat + ob_val_paf

        # calc gradient
        ob_val.backward()

        # take a step in gradient direction
        opt.step()

        # log the loss
        wandb.log({'loss': ob_val.item()}, step=global_step)
        global_step += 1

  0%|          | 0/450 [00:00<?, ?it/s]

run 1/5


  0%|          | 0/450 [00:00<?, ?it/s]

run 2/5


  0%|          | 0/450 [00:00<?, ?it/s]

run 3/5


 50%|████▉     | 224/450 [06:23<06:27,  1.71s/it]

In [ ]:
# print image
noise_resolution  = np.array([200, 400])
canvas = params_torch[diff_parameter].detach().cpu().numpy()
canvas = canvas.reshape(noise_resolution[1], noise_resolution[0], -1)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)
ax.imshow(canvas[:, :, [2, 1, 0]])
ax.axis('off')

In [ ]:
canvas_orig_list = []
canvas_noise_list = []
metric_list = []

for test_scene in tqdm(test_scenes):
    canvas_orig, canvas_noise, metric = get_image_and_score(test_scene, params_torch, body_estimation, diff_parameter)
    canvas_orig_list.append(canvas_orig)
    canvas_noise_list.append(canvas_noise)
    metric_list.append(metric)


In [ ]:
print(f"Avg metric: {np.mean(metric_list)}")

In [ ]:
plot_cnt = 16
for i in range(plot_cnt):
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(1, 2, 1)
    ax.imshow(canvas_orig_list[i])
    ax.axis('off')
    ax.set_title('Original')

    ax = fig.add_subplot(1, 2, 2)
    ax.imshow(canvas_noise_list[i])
    ax.axis('off')
    ax.set_title(f'Noise {metric_list[i]}')